# A tutorial on obtaining accurate speech-to-text alignment for long audio and noisy text for 1100+ languages

This tutorial consists of two parts.
- The first part corresponds to the paper *Less Peaky And More Accurate CTC Forced Alignment by Label Priors* published in ICASSP 2024. We will desmonstrate how to obtain more accurate speech-to-text alignment compared to a standard CTC model.
- In the second part, we will provide a robust pytorch-based speech-to-text alignment library ("TorchAudio Aligner") to align long audio and noisy text. We will replicate the data preparation step for preparing the [Librispeech corpus](https://www.openslr.org/12). For example, aligning the whole book, [Walden by Henry David Thoreau](https://www.gutenberg.org/cache/epub/205/pg205-images.html) (of 115K words), with its [audiobook chapter](https://librivox.org/walden-by-henry-david-thoreau/) (of 30 minutes in this demo, or even longer) in the [LibriVox project](https://librivox.org/).

20240417: This tutorial is still under development. This is the github repository: [https://github.com/huangruizhe/audio/tree/aligner_label_priors/examples/asr/librispeech_alignment](https://github.com/huangruizhe/audio/tree/aligner_label_priors/examples/asr/librispeech_alignment).

If you have any questions, please feel free to contact me (huangruizhe09@gmail.com)

## Preparation

In [ ]:
# Check python and pytorch's version
!python --version

# import necessary libraries
import torch
import torchaudio

print(f"PyTorch: {torch.__version__}")
print(f"TorchAudio: {torchaudio.__version__}")
print(f"CUDA: {torch.version.cuda}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import IPython
import sys
from tqdm import tqdm

For weighted finite state transducer (WFST), our library depends on [k2](https://github.com/k2-fsa/k2/), a pytorch-based WFST library.

In [ ]:
# This will take approximately 1~2 minutes

# k2 (cpu)
# pip install k2==1.24.4.dev20240223+cpu.torch2.2.1 -f https://k2-fsa.github.io/k2/cpu.html
# k2 (gpu)
!pip install k2==1.24.4.dev20240301+cuda12.1.torch2.2.1 -f https://k2-fsa.github.io/k2/cuda.html
!pip install pytorch-lightning
!pip install cmudict g2p_en
!pip install pydub
!pip install git+https://github.com/huangruizhe/lis.git

In [ ]:
![ ! -d "audio" ] && git clone -b aligner_label_priors https://github.com/huangruizhe/audio.git
sys.path.append("audio/examples/asr/librispeech_alignment")
sys.path.append("audio/examples/asr/librispeech_alignment/alignment")

## Part 1: obtaining more accurate CTC alignment by label priors

If you are not interested in Part 1, just go straight to Part 2. We have made the codes in Part 2 independent of Part 1.

### 1.1 Alignment with standard CTC model

#### 1.1.1 Training

The standard CTC model can be trained with the following command. As in the paper, the acoustic model is a TDNN-FFN model of 5M parameters. The frame rate is 0.02 seconds.
```
cd audio/examples/asr/librispeech_alignment/
exp_dir="experiments/ctc_tdnndnn_k2_0.0"

python train.py \
--exp-dir $exp_dir \   
--librispeech-path <path-to-librispeech> \
--global-stats-path ./global_stats.json \
--epochs 10 \
--nodes 1 \
--gpus 4
```

This will train a model in 10 epochs, with train/validation loss going from 3053/2394 down to 1990/1549.

#### 1.1.2 Decoding

The CTC model can be decoded with the following command. Note, the main purpose of the model is not for speech recognition. We just decode it with greedy algorithm as a sanity check. The character error rates (CER) on Librispeech are 4.98%/5.61% on test-clean and test-other.

```
python eval.py \
--checkpoint-path $exp_dir/checkpoints/epoch=9-step=65369.ckpt \
--librispeech-path <path-to-librispeech> \
--global-stats-path ./global_stats.json  \
--use-cuda
```

#### 1.1.3 Alignment

In [ ]:
SPEECH_FILE = torchaudio.utils.download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav")
transcript = "i had that curiosity beside me at this moment"

In [ ]:
IPython.display.Audio(SPEECH_FILE)

In [ ]:
from alignment.tokenizer import EnglishPhonemeTokenizer
from transforms import TestTransform
from lightning import AcousticModelModule
import IPython
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.figsize"] = [16.0, 3.0]

Prepare data for the acoustic model:

In [ ]:
tokenizer = EnglishPhonemeTokenizer()
global_stats_path = "audio/examples/asr/librispeech_alignment/global_stats.json"
test_transform = TestTransform(global_stats_path=global_stats_path, tokenizer=tokenizer)

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
speaker_id, chapter_id, utterance_id = 0, 0, 0

sample = (waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id)
batch, samples = test_transform(sample)

Load the acoustic model:

In [ ]:
checkpoint_path = "audio/examples/asr/librispeech_alignment/alignment/checkpoints/ctc_tdnndnn_k2_0.0/epoch=9-step=65369.ckpt"
model_standard = AcousticModelModule.load_from_checkpoint(checkpoint_path, tokenizer=tokenizer).eval()

Get the forced alignment:

In [ ]:
def unflatten(list_, lengths):
    assert len(list_) == sum(lengths)
    i = 0
    ret = []
    for l in lengths:
        ret.append(list_[i : i + l])
        i += l
    return ret

tokenized_transcript = tokenizer.encode_flatten(transcript, out_type=int)
emission1 = model_standard.forward(batch)
aligned_tokens1, alignment_scores1 = model_standard.align(batch, tokenized_transcript)
token_spans1 = torchaudio.functional.merge_tokens(aligned_tokens1, alignment_scores1)
word_spans1 = unflatten(token_spans1, [len(tokenizer.encode_flatten(word)) for word in transcript.split()])

Visualize the alignment:

In [ ]:
# Compute average score weighted by the span length
def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=0.02):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start + 0.1, t_spans[-1].end - 0.1
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    axes[1].set_xlim([0, None])
    fig.tight_layout()


plot_alignments(waveform, word_spans1, emission1, transcript.split())

In [ ]:
def plot_scores(word_spans, scores):
    fig, ax = plt.subplots()
    span_xs, span_hs = [], []
    ax.axvspan(word_spans[0][0].start - 0.05, word_spans[-1][-1].end + 0.05, facecolor="paleturquoise", edgecolor="none", zorder=-1)
    for t_span in word_spans:
        for span in t_span:
            for t in range(span.start, span.end):
                span_xs.append(t + 0.5)
                span_hs.append(scores[t].item())
            ax.annotate(tokenizer.id2token[span.token], (span.start, -0.07))
        ax.axvspan(t_span[0].start - 0.05, t_span[-1].end + 0.05, facecolor="mistyrose", edgecolor="none", zorder=-1)
    ax.bar(span_xs, span_hs, color="lightsalmon", edgecolor="coral")
    ax.set_title("Frame-level scores and word segments")
    ax.set_ylim(-0.1, None)
    ax.grid(True, axis="y")
    ax.axhline(0, color="black")
    fig.tight_layout()


plot_scores(word_spans1, alignment_scores1)

### 1.2 Alignment with CTC model with label priors

The CTC model with label priors can be trained with the following command. The only difference is that we added the "--alpha 0.6" option.
```
cd audio/examples/asr/librispeech_alignment/
exp_dir="experiments/ctc_tdnndnn_k2_0.6"

python train.py \
--exp-dir $exp_dir \   
--librispeech-path <path-to-librispeech> \
--global-stats-path ./global_stats.json \
--epochs 10 \
--nodes 1 \
--gpus 4 \
--alpha 0.6
```

This will train a model in 10 epochs, with the label priors for the blank token converging from 0.79 to 0.28.

The model can be decoded with the same command as above. The CER is 4.98%/5.61% on test-clean and test-other, same as the standard CTC model.

In [ ]:
checkpoint_path = "audio/examples/asr/librispeech_alignment/alignment/checkpoints/ctc_tdnndnn_k2_0.6/epoch=9-step=65369.ckpt"
model_with_priors = AcousticModelModule.load_from_checkpoint(checkpoint_path, tokenizer=tokenizer).eval()

In [ ]:
tokenized_transcript = tokenizer.encode_flatten(transcript, out_type=int)
emission2 = model_with_priors.forward(batch)
aligned_tokens2, alignment_scores2 = model_with_priors.align(batch, tokenized_transcript)
token_spans2 = torchaudio.functional.merge_tokens(aligned_tokens2, alignment_scores2)
word_spans2 = unflatten(token_spans2, [len(tokenizer.encode_flatten(word)) for word in transcript.split()])

In [ ]:
plot_alignments(waveform, word_spans2, emission2, transcript.split())

In [ ]:
plot_scores(word_spans2, alignment_scores2)

As we can see from the visualization above, the CTC model trained with label priors can produce less peaky posteriors compared to a standard CTC model. We can also listen to each word segments -- the alignments with label priors are more accurate than the standard one (e.g., the words "i", "me").

In [ ]:
def preview_word(waveform, spans, num_frames, transcript, sample_rate=16000):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    segment = waveform[:, x0:x1]
    return IPython.display.Audio(segment.numpy(), rate=sample_rate)


num_frames = emission1.size(1)

In [ ]:
# Listen to these words: 0, 5
word_idx = 5
preview_word(waveform, word_spans1[word_idx], num_frames, transcript.split()[word_idx], sample_rate)

In [ ]:
preview_word(waveform, word_spans2[word_idx], num_frames, transcript.split()[word_idx], sample_rate)

### 1.3 Fine-tuning CTC model with label priors

In [ ]:
# Coming soon! Basically, we will replicate the results in the paper in this part

## Part 2: obtaining robust alignment for long audio and noisy text

In part 1, we performed forced alignment at the utterance level. In practice, we don't usually have a small segment (e.g., 10 seconds) of audio and its corresponding exact, verbatim transcription as in a laboratory setting (e.g. [LIBRISPEECH](https://www.openslr.org/12) corpus). Instead, the audios come in long form (e.g., a whole mp3 recording of speech for an hour). The transcription for the whole recording can by noisy and non-verbatim, which may not exactly match what's been spoken in the recording. In particular, in order to use the raw speech data for machine learning, we usually need to prepare a corpus of segmented audios. In some applications such as automatic captioning, we still hope to align the long audio and text data as much as possible. In this tutorial, we will provide a python library, *TorchAudio Aligner*, to support such use cases.

Here, we are facing two challenges:
- **The audio is long**, which may not be suitable to be handled as a whole due to, e.g., limited CPU/GPU memory.
- **The transcript is long and noisy**. It can be a partial transcript with some missing words. It may have significant errors. It may also contain extra contents that's not spoken in the audio (e.g., the corresponding audio is corrupted). It can be a combination of all cases. Thus, the conventional, basic forced alignment algorithm could provide very bad alignment results, as it assumes the audio and text match exactly.

There are a few existing solutions:
- [Kaldi](https://ieeexplore.ieee.org/document/8268956), [Gentle](https://github.com/lowerquality/gentle) and [this work](https://ieeexplore.ieee.org/document/7404861) employ a weighted finite state transducer (WFST) framework to model the noisy texts.
- [WhisperX](https://github.com/m-bain/whisperX) uses attention mechanism to propose rough time stamps for uniformly segmented audio. Then, it performs phone-level or word-level forced alignment with an external aligner.
- [MMS](https://arxiv.org/abs/2305.13516) uses a special `<star>` token to handle missing words in the transcript.
- [SailAlign](https://www.semanticscholar.org/paper/SailAlign%3A-Robust-long-speech-text-alignment-Katsamanis-Georgiou/0b7f86429641b188cc62ec32eee590e8795a3d02) iteratively identifies reliable regions and then narrows down to align the remaining unaligned regions.

This tutorial is based on WFST and thus falls in the first category. Our implementation is based on PyTorch. Any CTC model in PyTorch can be equipped with our library to become a robust aligner. This makes our aligner distinguish from existing ones.

### 2.1 Preparation

In [ ]:
from alignment.torchaudio_k2_aligner import (
    uniform_segmentation_with_overlap,
    align_segments,
    concat_alignments,
    get_final_word_alignment,
    align,
    get_audacity_labels,
    get_gentle_visualization,
)
from alignment.tokenizer import EnglishCharTokenizer
from alignment.factor_transducer import make_factor_transducer_word_level_index_with_skip

We will use a pre-trained Wav2Vec2 model, [torchaudio.pipelines.MMS_FA](https://pytorch.org/audio/main/generated/torchaudio.pipelines.MMS_FA.html#torchaudio.pipelines.MMS_FA), as the acoustic model.

In [ ]:
bundle = torchaudio.pipelines.MMS_FA
model = bundle.get_model(with_star=False).to(device)

In [ ]:
LABELS = bundle.get_labels(star="*")
DICTIONARY = bundle.get_dict(star="*")

print(LABELS)

tokenizer = EnglishCharTokenizer(
    token2id=DICTIONARY,
    blk_token="-",
    unk_token="*",
)

### 2.2 Prepare long audio and noisy text

We will demonstrate aligning the whole book, [Walden by Henry David Thoreau](https://www.gutenberg.org/cache/epub/205/pg205-images.html) (of 115K words), with its audiobook chapter (of 30 minutes) in the [LibriVox project](https://librivox.org/walden-by-henry-david-thoreau/).

In [ ]:
# Download the whole book
import requests
from bs4 import BeautifulSoup

url = "https://www.gutenberg.org/cache/epub/205/pg205-images.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text = soup.get_text()
text = text.replace("\r\n", "\n")

In [ ]:
# Download a chapter of the audio book.
# The MP3 file is of 30 MB, and the download is usually very fast.
# You can check this out in your browser -- just go to Chapter 7:
# https://librivox.org/walden-by-henry-david-thoreau/
# But sometimes it can be also slow due to unknown network issue.
!wget https://ia800707.us.archive.org/20/items/walden_librivox/walden_c07.mp3

SPEECH_FILE = "walden_c07.mp3"

In [ ]:
# Play the long audio
# IPython.display.Audio(SPEECH_FILE)

In [ ]:
# Preview the transcript
print(text[:1000])

In [ ]:
# Preview the transcript relevant to the audio

# In this running example, the whole audio corresponds to
# text[271616: 293529]
# or
# text.split()[49489: 53362]

print(text[271400: 271400+1000])

As we can see above, the audio contains a header "This is a LibriVox recording ..." which is not transcribed. On the other hand, as we have downloaded the whole book, it contains a lot of extra text that's not spoken in the audio. Obviously, the standard forced alignment algorithm will not work in this case.

In [ ]:
# We tokenize the text into the labels of the acoustic model's output
text_tokenized = tokenizer.encode(tokenizer.text_normalize(text))

print(f"There are {len(text_tokenized)} words in the text")

# Preview the tokenization results. This corresponds to the beginning of the audio
print(tokenizer.decode(text_tokenized)[49489: 49489+15])

### 2.3 Use WFST to represent the text

#### 2.3.1 WFST basics

In [ ]:
# Coming soon!

#### 2.3.2 CTC graph, factor transducer and the variants

In [ ]:
# Coming soon!

Now, we will represent the whole book of over 100K words into one single WFST decoding graph that allows occasional insertion/deletioin/substitution errors. We set `skip_penalty=-0.5` and `return_penalty=-18.0`.

In [ ]:
# Get the decoding graph of the whole book

decoding_graph, word_index_sym_tab, token_sym_tab = \
    make_factor_transducer_word_level_index_with_skip(
        text_tokenized,
        blank_penalty=0,
        skip_penalty=-0.5,
        return_penalty=-18.0
    )
decoding_graph = decoding_graph.to(device)

print(f"There are {decoding_graph.shape[0]} nodes and {decoding_graph.num_arcs} arcs in the decoding graph for the text of {len(text_tokenized)} words.")
print(f"The decoding graph is on device: {decoding_graph.device}")

### 2.4 Handle long audio

We hope to feed the audio into the neural network to get the frame-wise posteriors (i.e., the emission matrix) over the label vocabulary. However, the audio book chapter is of about 30 minutes. This is too long to feed into the acoustic model all at once.

A common practice is to segment the long audio into small overlapping segments. The segments are processed independently and the results are concatenated appropriately to make the final alignment result. Here, as the Wav2Vec2 pretrained model that we use is taking a raw wave form as input, so we will segment the original audio into 15-second segments. In practice, we can also segment the feature vector (e.g., Fbanks) to feed into the acoustic model.

The library that we provide in this tutorial provides functions for appropriate segmentation and concatenation.

Now, let's use the library to segment the 30-minute audio into 15-second segments, with an overlap of 2 seconds between neighboring segments.

In [ ]:
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
resample_rate = 16000  # this is the sample rate of the Wav2Vec2 model
waveform = torchaudio.functional.resample(waveform, sample_rate, resample_rate)
sample_rate = resample_rate
print(waveform.shape, sample_rate)

In [ ]:
if waveform.dim() == 2:
    waveform.unsqueeze_(-1)
segment_size = sample_rate * 15 + 128  # 15 seconds; use extra 128 waveform samples to make sure we have 750 frames for each full-sized segment
overlap = sample_rate * 2 + 128        # 2 seconds
shortest_segment_size = sample_rate * 0.2  # if the last segment has less than 3200 samples (0.2 seconds), it will be discarded

waveform_segmented, segment_lengths, segment_offsets = uniform_segmentation_with_overlap(
    waveform,
    segment_size,
    overlap,
    shortest_segment_size=shortest_segment_size
)
waveform_segmented = waveform_segmented.squeeze()
print(waveform_segmented.shape, segment_lengths.shape, segment_offsets.shape)

We can listen to a segment to make sure the segmentation pipeline runs correctly.

In [ ]:
IPython.display.Audio(waveform_segmented[0], rate=sample_rate)

### 2.5 Obtain alignment

Now, we will feed the short segments to the neural network based acoustic model. We do this in batches. For each segment, the acoustic model produces frame-wise classification `batch_emissions` over its label vocabulary. This is then combined with the WFST `decoding_graph`, which is shared by all segments, to produce the best alignment path for each segment. If the `decoding_graph` is a linear WFST, which means there is no insertion/deletion/substitution errors in the transcript, this is equivalent to the conventional forced alignment as provided by this [TorchAudio API](https://pytorch.org/audio/main/tutorials/ctc_forced_alignment_api_tutorial.html).

There something to note for the `decoding_graph`. When we combine it with the neural network output, its "input label" should match the network's label vocabulary. However, it is our decision to define decoding graph's "output labels", which basically ask this question: what do we align the audio to? Here are two examples of design choices:

- The output labels are word labels or phoneme labels, as in most WFST-based alignment solutions (e.g., in [Gentle](https://github.com/lowerquality/gentle/blob/master/gentle/diff_align.py#L16)). In this case, the alignment paths consist of the word-level or phoneme-level transcripts predicted by the model.

- The output labels are the **word indices** in the transcript, instead of word labels. There are two benefits: (1) we can obtain the word labels easily from the word indices; (2) with word indices, we are able to preserve the ordering of words in the transcript, even though the alignment paths may contain words in the order different from the original transcript. As we will see later, this word ordering information enables efficient and effective heuristics for post-processing.

In [ ]:
if device == torch.device("cpu"):
    batch_size = 4
else:
    batch_size = 32

output_frames_offset = segment_offsets // (sample_rate * 0.02)

alignment_results = list()
for i in tqdm(range(0, waveform_segmented.size(0), batch_size)):
    batch_waveform_segmented = waveform_segmented[i: i+batch_size].to(device)
    batch_segment_lengths = segment_lengths[i: i+batch_size]
    batch_output_frames_offset = output_frames_offset[i: i+batch_size]

    with torch.inference_mode():
        # Checkout the API of the forward function here: https://github.com/pytorch/audio/blob/main/src/torchaudio/pipelines/_wav2vec2/utils.py#L34
        batch_emissions, batch_emissions_lengths = model(batch_waveform_segmented.to(device), batch_segment_lengths.to(device))

    # Attach the star dimension manually, see torchaudio issue #3772
    star_dim = torch.empty((batch_emissions.size(0), batch_emissions.size(1), 1), device=batch_emissions.device, dtype=batch_emissions.dtype)
    star_dim[:] = -5.0
    batch_emissions = torch.cat((batch_emissions, star_dim), 2)

    # `token_ids` and `timestamps` will each be a list of lists.
    # Each sublist corresponds to a segment in the batch.
    batch_results = align_segments(
        batch_emissions,
        decoding_graph,
        batch_emissions_lengths,
    )

    # The interpretation of `token.token_id` depends on the decoding graph.
    # Here, in this tutorial, `token.token_id` is the key to the `word_index_sym_tab``
    # and `token_sym_tab` dictionaries.
    for aligned_tokens, offset in zip(batch_results, batch_output_frames_offset):
        for token in aligned_tokens:
            token.timestamp += offset  # This will become the absolute frame timestamp in the whole audio
            if token.token_id == tokenizer.blk_id:
                continue
            if token.token_id in word_index_sym_tab:
                token.attr["wid"] = word_index_sym_tab[token.token_id]
            if token.token_id in token_sym_tab:
                token.attr["tk"] = token_sym_tab[token.token_id]

    alignment_results.extend(batch_results)

    # It will be too slow to do inference on CPU.
    # As it's only for the demonstration purpose, so we will break the loop here.
    # We can still see some partial alignment results.
    if device == torch.device("cpu"):
        break

We've got the alignment results for all 140 15-second segments in less than a minute, thanks to GPU acceleration.

In [ ]:
len(alignment_results)

Now we are going to concatenate the alignment results appropriately. We need to take care of two things: (1) the mis-aligned results, (2) the overlapping parts. The common algorithm for this concatenation step is through [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), which aligns the aligned transcript segments with the ground-truth transcript, e.g., as in [this codes in Gentle](https://github.com/lowerquality/gentle/blob/master/gentle/diff_align.py#L16). As we have word indices instead of word labels, we can do this a bit differently and better. Given the alignment results for all segments, we will find the [longest increasing subsequence (LIS)](https://en.wikipedia.org/wiki/Longest_increasing_subsequence) in the word indices. This can be done in $O(NlogN)$ time complexity and $O(N)$ space complexity, instead of $O(N^2)$ by Levenshtein distance, where $N$ is the total length of the segment-wise alignment results. This can be useful especially when $N$ is large for long audios. The python implementation of efficient LIS algorithm can be found in [this repository](https://github.com/huangruizhe/lis).

Moreover, with word indices, we are able to develop heuristics to remove outliers in the alignment results. This has been all implemented in our alignment library. E.g., by calling the `concat_alignments` function below.

In [ ]:
# Concatenate the alignment results for segments

# `resolved_alignment_results` is a list of `AlignedToken`
# `unaligned_text_indices` is a list of (start_word_index, end_word_index)
#    which corresponds to "holes" in the long text that are not aligned
resolved_alignment_results, unaligned_text_indices = concat_alignments(
    alignment_results,
    neighborhood_size=5,
)

len(resolved_alignment_results), len(unaligned_text_indices)

# We can check the results in `resolved_alignment_results`:
# text_splitted = text.split()
# [text_splitted[tk.attr['wid']] if 'wid' in tk.attr else tokenizer.id2token[tk.attr['tk']] for tk in resolved_alignment_results[-40:-1]]

Our final word-level alignment results can be obtained by the following, where `word_alignment` is a dictionary from word indices in `text` to an `AlignedWord` object.

In [ ]:
# `word_alignment` is a dict of word index in the long text => AlignedWord object
word_alignment = get_final_word_alignment(resolved_alignment_results, text, tokenizer)

Finally, let's preview the alignment results:

In [ ]:
list(word_alignment.items())[:10]

In [ ]:
list(word_alignment.items())[-10:]

As we can see from above, the alignment results look pretty good! Despite the book being long and noisy, it successfully locates [the chapter](https://www.gutenberg.org/cache/epub/205/pg205-images.html#chap08) from the whole book corresponding to the audio, except the first two words ("The Bean-Field") which are the chapter title. We will do some visualization to verify this results later.

On the other hand, in `unaligned_text_indices`, we can see the parts in the book chapter that are not successfully aligned to the audio:

In [ ]:
# The range [s, e] (both inclusive) is a "hole" in the long text that is not aligned
# where s and e are the word indices in the long text
text_splitted = text.split()
print(f"There are {len([None for s, e in unaligned_text_indices if e - s > 0])} holes of size greater than one word, out of {len(unaligned_text_indices)} holes in total.\n")
for s, e in unaligned_text_indices:
    if e - s > 0: # ignore the "holes" of only one word
        print(f"Unaligned text range: [{s}, {e}]")
        print(f"Unaligned text: {text_splitted[s:e+1]}")
        print(f"Time range: {word_alignment[s-1].start_time * 0.02:.2f} ~ {word_alignment[e+1].start_time * 0.02:.2f} seconds")
        print()

### 2.6 Visualization of alignment results

Finally, we can listen to and visually check the alignment results. We will provide two visualization with [Audacity](https://www.audacityteam.org/) and [Gentle](http://lowerquality.com/gentle/).

#### 2.6.1 Play a segment of the audio

To verify the alignment results, we can take a segment of the long audio and see if it matches its alignment.

In [ ]:
import random

# choose a random position in the word-level alignment results
i = random.randint(0, len(word_alignment) - 1)
word_alignment_segment = list(word_alignment.items())[i: i + 50]
t1 = word_alignment_segment[0][1].start_time.item() * 0.02
t2 = word_alignment_segment[-1][1].start_time.item() * 0.02
t1, t2

In [ ]:
text_segment = text_splitted[word_alignment_segment[0][0]: word_alignment_segment[-1][0]]
" ".join(text_segment)

In [ ]:
from pydub import AudioSegment

audio_segment = AudioSegment.from_file(SPEECH_FILE)
audio_segment = audio_segment[t1 * 1000: t2 * 1000]

IPython.display.Audio(audio_segment.get_array_of_samples(), rate=audio_segment.frame_rate)

We can see that the segmentation is pretty accurate based on the timestamps we obtained from the alignment!

#### 2.6.2 Audacity

For Audacity, we will provide the label file that can be imported to the software along with the audio file. Follow the instructions [here](https://manual.audacityteam.org/man/importing_and_exporting_labels.html) to import labels.

<img src="https://i.postimg.cc/3wpFfSJV/Screen-Shot-2024-04-18-at-4-40-44-AM.jpg" width="600">

In [ ]:
audacity_labels_str = get_audacity_labels(word_alignment, frame_duration=0.02)

audacity_path = str(SPEECH_FILE)[:-4] + "_audacity.txt"
with open(audacity_path, "w") as fout:
    print(audacity_labels_str, file=fout)

#### 2.6.3 Gentle

Gentle comes with a nice, handy [visualization mechanism](https://github.com/lowerquality/gentle/blob/master/serve.py#L113) based on HTML and JavaScript. The following function will generate the necessary files (the "index.html" file) and instructions for visualization in Gentle.

<img src="https://i.postimg.cc/xd3rLCQk/Screen-Shot-2024-04-18-at-4-34-40-AM.jpg" width="600">

In [ ]:
get_gentle_visualization(word_alignment, tokenizer, frame_duration=0.02, audio_file=SPEECH_FILE, text=text, i_word_start=49489, i_word_end=53362)

### 2.7 Multilingual support

According to [this tutorial](https://pytorch.org/audio/stable/tutorials/forced_alignment_for_multilingual_data_tutorial.html), we can use the same MMS_FA model that we have used above to align long audio and noisy text in other languages -- MMS_FA supports 1100+ languages! Other multiligual models, such as [XLSR](https://huggingface.co/masoudmzb/wav2vec2-xlsr-multilingual-53-fa), can be considered as well. Let's demonstrate this with some examples. We choose books in other languages from [LibriVox](https://librivox.org/search?primary_key=0&search_category=language&search_page=1&search_form=get_results&search_order=alpha). We will align audio to the book. In the end, we will listen to the alignment segments to see if they are aligned correctly.

#### 2.7.1 Portuguese

We take this chapter as the example: "46 Ode Triunfal" in [Orpheu no.1](https://librivox.org/orpheu-no1/). This audio book chapter is of 17 minutes long. The [book](https://www.gutenberg.org/cache/epub/23620/pg23620-images.html) has about 18K words.

In [ ]:
# Download audio

!wget https://ia801705.us.archive.org/7/items/orpheu_no1_2010_librivox/orpheuno1_46__128kb.mp3
SPEECH_FILE = "orpheuno1_46__128kb.mp3"

In [ ]:
# Download text

url = "https://www.gutenberg.org/cache/epub/23620/pg23620-images.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text = soup.get_text()
text = text.replace("\r\n", "\n")

In [ ]:
# Preview text

print(text[10000:11000])

In [ ]:
!pip install uroman-python

In [ ]:
# Tokenization

import uroman
import re

def normalize_uroman(text):
    text = text.lower()
    text = text.replace("’", "'")
    text = re.sub("([^a-z' ])", " ", text)
    text = re.sub(' +', ' ', text)
    return text.strip()

text_romanized = uroman.uroman(text, language="por")
text_normalized = normalize_uroman(text_romanized)
text_tokenized = tokenizer.encode(tokenizer.text_normalize(text_normalized))

print(f"There are {len(text_tokenized)} words in the text")

# Preview the tokenization results. This corresponds to the beginning of the audio
print(tokenizer.decode(text_tokenized)[14444: 14444+15])

In [ ]:
# Get the decoding graph of the whole book

decoding_graph, word_index_sym_tab, token_sym_tab = \
    make_factor_transducer_word_level_index_with_skip(
        text_tokenized,
        blank_penalty=0,
        skip_penalty=-0.5,
        return_penalty=-18.0
    )
decoding_graph = decoding_graph.to(device)

print(f"There are {decoding_graph.shape[0]} nodes and {decoding_graph.num_arcs} arcs in the decoding graph for the text of {len(text_tokenized)} words.")
print(f"The decoding graph is on device: {decoding_graph.device}")

In [ ]:
# Handle long audio

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
resample_rate = 16000  # this is the sample rate of the Wav2Vec2 model
waveform = torchaudio.functional.resample(waveform, sample_rate, resample_rate)
sample_rate = resample_rate
print(waveform.shape, sample_rate)

In [ ]:
# Uniform segmentation

if waveform.dim() == 2:
    waveform.unsqueeze_(-1)
segment_size = sample_rate * 15 + 128  # 15 seconds; use extra 128 waveform samples to make sure we have 750 frames for each full-sized segment
overlap = sample_rate * 2 + 128        # 2 seconds
shortest_segment_size = sample_rate * 0.2  # if the last segment has less than 3200 samples (0.2 seconds), it will be discarded

waveform_segmented, segment_lengths, segment_offsets = uniform_segmentation_with_overlap(
    waveform,
    segment_size,
    overlap,
    shortest_segment_size=shortest_segment_size
)
waveform_segmented = waveform_segmented.squeeze()
print(waveform_segmented.shape, segment_lengths.shape, segment_offsets.shape)

In [ ]:
IPython.display.Audio(waveform_segmented[0], rate=sample_rate)

In [ ]:
# Obtain alignment for segments

if device == torch.device("cpu"):
    batch_size = 4
else:
    batch_size = 32

output_frames_offset = segment_offsets // (sample_rate * 0.02)

alignment_results = list()
for i in tqdm(range(0, waveform_segmented.size(0), batch_size)):
    batch_waveform_segmented = waveform_segmented[i: i+batch_size].to(device)
    batch_segment_lengths = segment_lengths[i: i+batch_size]
    batch_output_frames_offset = output_frames_offset[i: i+batch_size]

    with torch.inference_mode():
        # Checkout the API of the forward function here: https://github.com/pytorch/audio/blob/main/src/torchaudio/pipelines/_wav2vec2/utils.py#L34
        batch_emissions, batch_emissions_lengths = model(batch_waveform_segmented.to(device), batch_segment_lengths.to(device))

    # Attach the star dimension manually, see torchaudio issue #3772
    star_dim = torch.empty((batch_emissions.size(0), batch_emissions.size(1), 1), device=batch_emissions.device, dtype=batch_emissions.dtype)
    star_dim[:] = -5.0
    batch_emissions = torch.cat((batch_emissions, star_dim), 2)

    # `token_ids` and `timestamps` will each be a list of lists.
    # Each sublist corresponds to a segment in the batch.
    batch_results = align_segments(
        batch_emissions,
        decoding_graph,
        batch_emissions_lengths,
    )

    # The interpretation of `token.token_id` depends on the decoding graph.
    # Here, in this tutorial, `token.token_id` is the key to the `word_index_sym_tab``
    # and `token_sym_tab` dictionaries.
    for aligned_tokens, offset in zip(batch_results, batch_output_frames_offset):
        for token in aligned_tokens:
            token.timestamp += offset  # This will become the absolute frame timestamp in the whole audio
            if token.token_id == tokenizer.blk_id:
                continue
            if token.token_id in word_index_sym_tab:
                token.attr["wid"] = word_index_sym_tab[token.token_id]
            if token.token_id in token_sym_tab:
                token.attr["tk"] = token_sym_tab[token.token_id]

    alignment_results.extend(batch_results)

    # It will be too slow to do inference on CPU.
    # As it's only for the demonstration purpose, so we will break the loop here.
    # We can still see some partial alignment results.
    if device == torch.device("cpu"):
        break

In [ ]:
# Concatenate the alignment results for segments

# `resolved_alignment_results` is a list of `AlignedToken`
# `unaligned_text_indices` is a list of (start_word_index, end_word_index)
#    which corresponds to "holes" in the long text that are not aligned
resolved_alignment_results, unaligned_text_indices = concat_alignments(
    alignment_results,
    neighborhood_size=5,
)

len(resolved_alignment_results), len(unaligned_text_indices)

In [ ]:
# Get the final word-level alignments

# `word_alignment` is a dict of word index in the long text => AlignedWord object
word_alignment = get_final_word_alignment(resolved_alignment_results, text_normalized, tokenizer)

In [ ]:
import random

# choose a random position in the word-level alignment results
i = random.randint(0, len(word_alignment) - 1)
word_alignment_segment = list(word_alignment.items())[i: i + 50]
t1 = word_alignment_segment[0][1].start_time.item() * 0.02
t2 = word_alignment_segment[-1][1].start_time.item() * 0.02
t1, t2

In [ ]:
text_splitted = text_normalized.split()
text_segment = text_splitted[word_alignment_segment[0][0]: word_alignment_segment[-1][0]]
" ".join(text_segment)

In [ ]:
from pydub import AudioSegment

audio_segment = AudioSegment.from_file(SPEECH_FILE)
audio_segment = audio_segment[t1 * 1000: t2 * 1000]

IPython.display.Audio(audio_segment.get_array_of_samples(), rate=audio_segment.frame_rate)

#### 2.7.2 Chinese

We take this chapter as the example: "14 憲問， 第十四" in [論語 Lun Yu (Analects of Confucius)](https://librivox.org/lun-yu-or-analects-of-confucius-read-in-chinese/). This audio book chapter is of 11.5 minutes long. The whole book has about 15K Chinese characters.

In [ ]:
# Download audio

!wget https://ia801307.us.archive.org/15/items/lun_yu_0801_librivox/lunyu_14_confucius.mp3
SPEECH_FILE = "lunyu_14_confucius.mp3"

In [ ]:
# Download text

![ ! -d "The-Analects" ] && git clone https://github.com/Muzi828/The-Analects.git

with open('The-Analects/论语-原文.txt', 'r', encoding='gbk') as f:
    # Read the entire file as a string
    text = f.read()
text = text.replace("\r\n", "\n")

In [ ]:
# Preview text

print(text[:200])

In [ ]:
!pip install zhon uroman-python

In [ ]:
# Tokenization

import uroman
import re
import zhon
import string

punctuation_set = set(zhon.hanzi.punctuation + string.punctuation)

def remove_punctuation_and_split_characters(text):
    text = "".join(text.split())
    text = "".join([c for c in text if c not in punctuation_set])
    return " ".join(list(text))

text_normalized = remove_punctuation_and_split_characters(text)
text_romanized = uroman.uroman(text_normalized, language="cmn")
text_tokenized = tokenizer.encode(tokenizer.text_normalize(text_romanized))

assert len(text_normalized.split()) == len(text_romanized.split()) == len(text_tokenized)

print(f"There are {len(text_tokenized)} words in the text")

# Preview the tokenization results.
print(list(zip(tokenizer.decode(text_tokenized), text_normalized.split()))[144: 144+15])

In [ ]:
# Get the decoding graph of the whole book

decoding_graph, word_index_sym_tab, token_sym_tab = \
    make_factor_transducer_word_level_index_with_skip(
        text_tokenized,
        blank_penalty=0,
        skip_penalty=-0.5,
        return_penalty=-18.0
    )
decoding_graph = decoding_graph.to(device)

print(f"There are {decoding_graph.shape[0]} nodes and {decoding_graph.num_arcs} arcs in the decoding graph for the text of {len(text_tokenized)} words.")
print(f"The decoding graph is on device: {decoding_graph.device}")

In [ ]:
# Handle long audio

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
resample_rate = 16000  # this is the sample rate of the Wav2Vec2 model
waveform = torchaudio.functional.resample(waveform, sample_rate, resample_rate)
sample_rate = resample_rate
print(waveform.shape, sample_rate)

In [ ]:
# Uniform segmentation

if waveform.dim() == 2:
    waveform.unsqueeze_(-1)
segment_size = sample_rate * 15 + 128  # 15 seconds; use extra 128 waveform samples to make sure we have 750 frames for each full-sized segment
overlap = sample_rate * 2 + 128        # 2 seconds
shortest_segment_size = sample_rate * 0.2  # if the last segment has less than 3200 samples (0.2 seconds), it will be discarded

waveform_segmented, segment_lengths, segment_offsets = uniform_segmentation_with_overlap(
    waveform,
    segment_size,
    overlap,
    shortest_segment_size=shortest_segment_size
)
waveform_segmented = waveform_segmented.squeeze()
print(waveform_segmented.shape, segment_lengths.shape, segment_offsets.shape)

In [ ]:
IPython.display.Audio(waveform_segmented[0], rate=sample_rate)

In [ ]:
# Obtain alignment for segments

if device == torch.device("cpu"):
    batch_size = 4
else:
    batch_size = 32

output_frames_offset = segment_offsets // (sample_rate * 0.02)

alignment_results = list()
for i in tqdm(range(0, waveform_segmented.size(0), batch_size)):
    batch_waveform_segmented = waveform_segmented[i: i+batch_size].to(device)
    batch_segment_lengths = segment_lengths[i: i+batch_size]
    batch_output_frames_offset = output_frames_offset[i: i+batch_size]

    with torch.inference_mode():
        # Checkout the API of the forward function here: https://github.com/pytorch/audio/blob/main/src/torchaudio/pipelines/_wav2vec2/utils.py#L34
        batch_emissions, batch_emissions_lengths = model(batch_waveform_segmented.to(device), batch_segment_lengths.to(device))

    # Attach the star dimension manually, see torchaudio issue #3772
    star_dim = torch.empty((batch_emissions.size(0), batch_emissions.size(1), 1), device=batch_emissions.device, dtype=batch_emissions.dtype)
    star_dim[:] = -5.0
    batch_emissions = torch.cat((batch_emissions, star_dim), 2)

    # `token_ids` and `timestamps` will each be a list of lists.
    # Each sublist corresponds to a segment in the batch.
    batch_results = align_segments(
        batch_emissions,
        decoding_graph,
        batch_emissions_lengths,
    )

    # The interpretation of `token.token_id` depends on the decoding graph.
    # Here, in this tutorial, `token.token_id` is the key to the `word_index_sym_tab``
    # and `token_sym_tab` dictionaries.
    for aligned_tokens, offset in zip(batch_results, batch_output_frames_offset):
        for token in aligned_tokens:
            token.timestamp += offset  # This will become the absolute frame timestamp in the whole audio
            if token.token_id == tokenizer.blk_id:
                continue
            if token.token_id in word_index_sym_tab:
                token.attr["wid"] = word_index_sym_tab[token.token_id]
            if token.token_id in token_sym_tab:
                token.attr["tk"] = token_sym_tab[token.token_id]

    alignment_results.extend(batch_results)

    # It will be too slow to do inference on CPU.
    # As it's only for the demonstration purpose, so we will break the loop here.
    # We can still see some partial alignment results.
    if device == torch.device("cpu"):
        break

In [ ]:
# Concatenate the alignment results for segments

# `resolved_alignment_results` is a list of `AlignedToken`
# `unaligned_text_indices` is a list of (start_word_index, end_word_index)
#    which corresponds to "holes" in the long text that are not aligned
resolved_alignment_results, unaligned_text_indices = concat_alignments(
    alignment_results,
    neighborhood_size=5,
)

len(resolved_alignment_results), len(unaligned_text_indices)

In [ ]:
# Get the final word-level alignments

# `word_alignment` is a dict of word index in the long text => AlignedWord object
word_alignment = get_final_word_alignment(resolved_alignment_results, text_normalized, tokenizer)

In [ ]:
import random

# choose a random position in the word-level alignment results
i = random.randint(0, len(word_alignment) - 1)
word_alignment_segment = list(word_alignment.items())[i: i + 50]
t1 = word_alignment_segment[0][1].start_time.item() * 0.02
t2 = word_alignment_segment[-1][1].start_time.item() * 0.02
t1, t2

In [ ]:
text_splitted = text_normalized.split()
text_segment = text_splitted[word_alignment_segment[0][0]: word_alignment_segment[-1][0]]
" ".join(text_segment)

In [ ]:
from pydub import AudioSegment

audio_segment = AudioSegment.from_file(SPEECH_FILE)
audio_segment = audio_segment[t1 * 1000: t2 * 1000]

IPython.display.Audio(audio_segment.get_array_of_samples(), rate=audio_segment.frame_rate)

#### 2.7.3 Japanese

We take this chapter as the example: "03 風立ちぬ" in [風立ちぬ (Kaze Tachinu)
](https://librivox.org/kaze-tachinu-by-hori-tatsuo/). This audio book chapter is of 57.5 minutes long. The [whole book](https://www.aozora.gr.jp/cards/001030/files/4803_14204.html) has about 57K Japanese characters.

In [ ]:
# Download audio

!wget https://ia803207.us.archive.org/30/items/kazetachinu_ek_librivox/kazetachinu_03_hori.mp3
SPEECH_FILE = "kazetachinu_03_hori.mp3"

In [ ]:
# Download text

# We need to parse a webpage in Japanese. We need to resolve the encoding correctly
# https://github.com/TokyoPython/decodeNihongo/tree/master

def retrieveURL(URL):
    import urllib.request
    import urllib.parse
    response = None
    reqfull = urllib.request.Request(URL)  #create Post request
    try:
        response = urllib.request.urlopen(reqfull)
    except urllib.request.HTTPError as err:
        if err.code == 404:
            print("URL not found 404 " + str(URL))
        else:
            print("URL not found " + str(err.code)+" "+ str(URL))
    return response

def get_decodedNihongo_PageText(URL):  #various encodings of Japanese-language web pages
    import html  #To unescape webpage content
    # Transmit request, and read first page of results
    response = retrieveURL(URL)
    if response != None:
        responseHTML = response.read()
        try:
            text = responseHTML.decode('utf-8')  #standard (utf-8 encoding)
        except:
            try:
                text = responseHTML.decode('shiftjis')  #most likely alternate Japanese encoding
            except:
                text = responseHTML.decode('shift_jisx0213') #be prepared for other Japanese encodings
        text = html.unescape(text)  #get rid of "&quot:", "&lt;", "&gt;",etc.
    return text


response_text = get_decodedNihongo_PageText("https://www.aozora.gr.jp/cards/001030/files/4803_14204.html")
soup = BeautifulSoup(response_text, "html.parser")

text = soup.get_text()
text = text.replace("\r\n", "\n")
text[:1000]

In [ ]:
# For Japanese romanization
!pip install cutlet unidic-lite zhon

In [ ]:
# Tokenization

import re
import zhon
import string
import cutlet

punctuation_set = set(zhon.hanzi.punctuation + string.punctuation)

def remove_punctuation_and_split_characters(text):
    text = "".join(text.split())
    text = "".join([c for c in text if c not in punctuation_set])
    return " ".join(list(text))

katsu = cutlet.Cutlet()

text_normalized = remove_punctuation_and_split_characters(text)
text_romanized = [katsu.romaji(w) for w in text_normalized.split()]
text_romanized = " ".join([w if w else "*" for w in text_romanized])
text_tokenized = tokenizer.encode(tokenizer.text_normalize(text_romanized))

assert len(text_normalized.split()) == len(text_romanized.split()) == len(text_tokenized)

print(f"There are {len(text_tokenized)} words in the text")

# Preview the tokenization results.
print(list(zip(tokenizer.decode(text_tokenized), text_normalized.split()))[144: 144+15])

In [ ]:
# Get the decoding graph of the whole book

decoding_graph, word_index_sym_tab, token_sym_tab = \
    make_factor_transducer_word_level_index_with_skip(
        text_tokenized,
        blank_penalty=0,
        skip_penalty=-0.5,
        return_penalty=-18.0
    )
decoding_graph = decoding_graph.to(device)

print(f"There are {decoding_graph.shape[0]} nodes and {decoding_graph.num_arcs} arcs in the decoding graph for the text of {len(text_tokenized)} words.")
print(f"The decoding graph is on device: {decoding_graph.device}")

In [ ]:
# Handle long audio

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
resample_rate = 16000  # this is the sample rate of the Wav2Vec2 model
waveform = torchaudio.functional.resample(waveform, sample_rate, resample_rate)
sample_rate = resample_rate
print(waveform.shape, sample_rate)

In [ ]:
# Uniform segmentation

if waveform.dim() == 2:
    waveform.unsqueeze_(-1)
segment_size = sample_rate * 15 + 128  # 15 seconds; use extra 128 waveform samples to make sure we have 750 frames for each full-sized segment
overlap = sample_rate * 2 + 128        # 2 seconds
shortest_segment_size = sample_rate * 0.2  # if the last segment has less than 3200 samples (0.2 seconds), it will be discarded

waveform_segmented, segment_lengths, segment_offsets = uniform_segmentation_with_overlap(
    waveform,
    segment_size,
    overlap,
    shortest_segment_size=shortest_segment_size
)
waveform_segmented = waveform_segmented.squeeze()
print(waveform_segmented.shape, segment_lengths.shape, segment_offsets.shape)

In [ ]:
IPython.display.Audio(waveform_segmented[0], rate=sample_rate)

In [ ]:
# Obtain alignment for segments

if device == torch.device("cpu"):
    batch_size = 4
else:
    batch_size = 32

output_frames_offset = segment_offsets // (sample_rate * 0.02)

alignment_results = list()
for i in tqdm(range(0, waveform_segmented.size(0), batch_size)):
    batch_waveform_segmented = waveform_segmented[i: i+batch_size].to(device)
    batch_segment_lengths = segment_lengths[i: i+batch_size]
    batch_output_frames_offset = output_frames_offset[i: i+batch_size]

    with torch.inference_mode():
        # Checkout the API of the forward function here: https://github.com/pytorch/audio/blob/main/src/torchaudio/pipelines/_wav2vec2/utils.py#L34
        batch_emissions, batch_emissions_lengths = model(batch_waveform_segmented.to(device), batch_segment_lengths.to(device))

    # Attach the star dimension manually, see torchaudio issue #3772
    star_dim = torch.empty((batch_emissions.size(0), batch_emissions.size(1), 1), device=batch_emissions.device, dtype=batch_emissions.dtype)
    star_dim[:] = -5.0
    batch_emissions = torch.cat((batch_emissions, star_dim), 2)

    # `token_ids` and `timestamps` will each be a list of lists.
    # Each sublist corresponds to a segment in the batch.
    batch_results = align_segments(
        batch_emissions,
        decoding_graph,
        batch_emissions_lengths,
    )

    # The interpretation of `token.token_id` depends on the decoding graph.
    # Here, in this tutorial, `token.token_id` is the key to the `word_index_sym_tab``
    # and `token_sym_tab` dictionaries.
    for aligned_tokens, offset in zip(batch_results, batch_output_frames_offset):
        for token in aligned_tokens:
            token.timestamp += offset  # This will become the absolute frame timestamp in the whole audio
            if token.token_id == tokenizer.blk_id:
                continue
            if token.token_id in word_index_sym_tab:
                token.attr["wid"] = word_index_sym_tab[token.token_id]
            if token.token_id in token_sym_tab:
                token.attr["tk"] = token_sym_tab[token.token_id]

    alignment_results.extend(batch_results)

    # It will be too slow to do inference on CPU.
    # As it's only for the demonstration purpose, so we will break the loop here.
    # We can still see some partial alignment results.
    if device == torch.device("cpu"):
        break

In [ ]:
# Concatenate the alignment results for segments

# `resolved_alignment_results` is a list of `AlignedToken`
# `unaligned_text_indices` is a list of (start_word_index, end_word_index)
#    which corresponds to "holes" in the long text that are not aligned
resolved_alignment_results, unaligned_text_indices = concat_alignments(
    alignment_results,
    neighborhood_size=5,
)

len(resolved_alignment_results), len(unaligned_text_indices)

In [ ]:
# Get the final word-level alignments

# `word_alignment` is a dict of word index in the long text => AlignedWord object
word_alignment = get_final_word_alignment(resolved_alignment_results, text_normalized, tokenizer)

In [ ]:
import random

# choose a random position in the word-level alignment results
i = random.randint(0, len(word_alignment) - 1)
word_alignment_segment = list(word_alignment.items())[i: i + 50]
t1 = word_alignment_segment[0][1].start_time.item() * 0.02
t2 = word_alignment_segment[-1][1].start_time.item() * 0.02
t1, t2

In [ ]:
text_splitted = text_normalized.split()
text_segment = text_splitted[word_alignment_segment[0][0]: word_alignment_segment[-1][0]]
" ".join(text_segment)

In [ ]:
from pydub import AudioSegment

audio_segment = AudioSegment.from_file(SPEECH_FILE)
audio_segment = audio_segment[t1 * 1000: t2 * 1000]

IPython.display.Audio(audio_segment.get_array_of_samples(), rate=audio_segment.frame_rate)

## TODOs:

*   Xiaohui suggested to demonstrate the mechanism to support ins/del/sub errors
*   Xiaohui suggested to save the unaligned audio/text and misaligned parts in the final results.
*   Let's make an API for multi-lingual support

